# Selector Group Chat  

`SelectorGroupChat` is a dynamic team-based chat system where participants take turns broadcasting messages to all other members. Unlike traditional round-robin approaches, it leverages a generative model (e.g., an LLM) to intelligently select the next speaker based on the ongoing conversation, enabling more context-aware and adaptive interactions.  

## Key Features  

- **Intelligent Speaker Selection** – A model-driven approach determines the most relevant next speaker.  
- **Configurable Roles & Descriptions** – Define unique roles and attributes for each participant.  
- **Turn Management** – Prevents consecutive turns by the same speaker (optional).  
- **Flexible Selection Process** – Customize the selection prompt and override the model’s decision with a custom function.  

## How It Works  

`SelectorGroupChat` builds upon the structure of `RoundRobinGroupChat` but introduces a model-based mechanism for selecting the next speaker. When a task is initiated through `run()` or `run_stream()`, the following steps occur:  

### 1. Context Analysis & Speaker Selection  
   - The system evaluates the conversation history and participant attributes (e.g., name, description).  
   - A model determines the most suitable next speaker.  
   - By default, the same speaker is not selected twice in a row unless they are the only available agent. This behavior can be modified using `allow_repeated_speaker=True`.  
   - You can also override the model-based selection by providing a custom selection function.  

### 2. Generating & Broadcasting the Response  
   - The selected speaker generates a response.  
   - The response is shared with all other participants.  

### 3. Checking for Termination  
   - The system evaluates whether the conversation should end based on predefined termination conditions.  
   - If the conversation is still active, the process repeats from Step 1.  

### 4. Returning the Final Result  
   - Once the conversation concludes, the system returns a `TaskResult` containing the complete conversation history.  

This approach ensures an adaptive, context-driven collaboration experience, making `SelectorGroupChat` ideal for scenarios requiring structured yet flexible interactions.  

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

def weather_check_tool(city: str) -> str:
    weather_data = {
        "Dubai": "Sunny, 35°C",
        "New York": "Cloudy, 22°C",
        "London": "Rainy, 18°C",
        "Tokyo": "Clear, 26°C"
    }
    return weather_data.get(city, "Weather data not available.")

def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        ("USD", "EUR"): 0.92,
        ("EUR", "USD"): 1.08,
        ("USD", "AED"): 3.67,
        ("AED", "USD"): 0.27
    }
    rate = exchange_rates.get((from_currency, to_currency), None)
    if rate:
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
    return "Exchange rate not available."



model_client = OpenAIChatCompletionClient(
    model="gpt-4o"
)


In [2]:

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks. It should break down tasks and delegate them to the appropriate agents.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WeatherAgent: Checks weather conditions
        CurrencyAgent: Handles currency conversion
    
    You only plan and delegate tasks - you do not execute them yourself.
    
    When assigning tasks, use this format:
    1. <agent> : <task>
    
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

weather_agent = AssistantAgent(
    "WeatherAgent",
    description="An agent that provides current weather conditions for a given city.",
    tools=[weather_check_tool],
    model_client=model_client,
    system_message="""
    You are a weather-checking agent.
    Your only tool is weather_check_tool - use it to fetch weather data for a city.
    """,
)

currency_agent = AssistantAgent(
    "CurrencyAgent",
    description="An agent that performs currency exchange calculations.",
    model_client=model_client,
    tools=[currency_exchange_tool],
    system_message="""
    You are a currency exchange agent.
    Your job is to convert a given amount from one currency to another using the available exchange rates.
    """,
)


In [5]:
# Mention all termination conditions

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [6]:
# Create the team

team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [ ]:
# Run the task
task = "What is the current weather in Dubai, and how much is 100 USD in AED?"
await Console(team.run_stream(task=task))

---------- user ----------
What is the current weather in Dubai, and how much is 100 USD in AED?
---------- WeatherAgent ----------
[FunctionCall(id='call_rXbJuY14BJNCJpV5sjQ6dTiT', arguments='{"city": "Dubai"}', name='weather_check_tool')]
---------- WeatherAgent ----------
[FunctionExecutionResult(content='Sunny, 35°C', call_id='call_rXbJuY14BJNCJpV5sjQ6dTiT', is_error=False)]
---------- WeatherAgent ----------
Sunny, 35°C
---------- CurrencyAgent ----------
[FunctionCall(id='call_8eyFQ3NN5sA0G6uCIJLwuSgN', arguments='{"amount":100,"from_currency":"USD","to_currency":"AED"}', name='currency_exchange_tool')]
---------- CurrencyAgent ----------
[FunctionExecutionResult(content='100.0 USD is equal to 367.00 AED.', call_id='call_8eyFQ3NN5sA0G6uCIJLwuSgN', is_error=False)]
---------- CurrencyAgent ----------
100.0 USD is equal to 367.00 AED.
---------- PlanningAgent ----------
1. WeatherAgent: Check the current weather in Dubai.
2. CurrencyAgent: Convert 100 USD to AED.

Summary:
- The cu

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the current weather in Dubai, and how much is 100 USD in AED?', type='TextMessage'), ToolCallRequestEvent(source='WeatherAgent', models_usage=RequestUsage(prompt_tokens=93, completion_tokens=32), content=[FunctionCall(id='call_rXbJuY14BJNCJpV5sjQ6dTiT', arguments='{"city": "Dubai"}', name='weather_check_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WeatherAgent', models_usage=None, content=[FunctionExecutionResult(content='Sunny, 35°C', call_id='call_rXbJuY14BJNCJpV5sjQ6dTiT', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='WeatherAgent', models_usage=None, content='Sunny, 35°C', type='ToolCallSummaryMessage'), ToolCallRequestEvent(source='CurrencyAgent', models_usage=RequestUsage(prompt_tokens=129, completion_tokens=26), content=[FunctionCall(id='call_8eyFQ3NN5sA0G6uCIJLwuSgN', arguments='{"amount":100,"from_currency":"USD","to_currency":"AED"}',

## Custom Selector Function in Multi-Agent System
In a multi-agent system, we sometimes need more control over the agent selection process. The `selector_func` allows us to override the default model-based selection with custom logic.

#### Use Case
We want the Planning Agent to respond immediately after any specialized agent (WeatherAgent or CurrencyAgent) has spoken, ensuring smooth coordination.

In [8]:
# Custom selector function

from typing import Sequence

def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


> <b>Note:</b> If the last message is not from the PlanningAgent, it forces the next response from PlanningAgent.
Otherwise, it allows normal selection.

In [9]:
await team.reset()
team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

In [10]:
task = "What is the current weather in Dubai, and how much is 100 USD in AED?"
await Console(team.run_stream(task=task))

---------- user ----------
What is the current weather in Dubai, and how much is 100 USD in AED?
---------- PlanningAgent ----------
1. WeatherAgent : Check the current weather conditions in Dubai.
2. CurrencyAgent : Convert 100 USD to AED using the latest exchange rate.
---------- WeatherAgent ----------
[FunctionCall(id='call_EQ12OhCIOjG8yYooKmiesIGV', arguments='{"city":"Dubai"}', name='weather_check_tool')]
---------- WeatherAgent ----------
[FunctionExecutionResult(content='Sunny, 35°C', call_id='call_EQ12OhCIOjG8yYooKmiesIGV', is_error=False)]
---------- WeatherAgent ----------
Sunny, 35°C
---------- PlanningAgent ----------
3. CurrencyAgent : Convert 100 USD to AED using the latest exchange rate.
---------- CurrencyAgent ----------
[FunctionCall(id='call_rjmvz4pIKxWZcYBmyuEvH7iO', arguments='{"amount":100,"from_currency":"USD","to_currency":"AED"}', name='currency_exchange_tool')]
---------- CurrencyAgent ----------
[FunctionExecutionResult(content='100.0 USD is equal to 367.00 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the current weather in Dubai, and how much is 100 USD in AED?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=132, completion_tokens=31), content='1. WeatherAgent : Check the current weather conditions in Dubai.\n2. CurrencyAgent : Convert 100 USD to AED using the latest exchange rate.', type='TextMessage'), ToolCallRequestEvent(source='WeatherAgent', models_usage=RequestUsage(prompt_tokens=130, completion_tokens=16), content=[FunctionCall(id='call_EQ12OhCIOjG8yYooKmiesIGV', arguments='{"city":"Dubai"}', name='weather_check_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WeatherAgent', models_usage=None, content=[FunctionExecutionResult(content='Sunny, 35°C', call_id='call_EQ12OhCIOjG8yYooKmiesIGV', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='WeatherAgent', models_usage=None, content='Sunny, 35°